In [4]:

import numpy as np
import random
import matplotlib.pyplot as plt
import math
import time
import scipy  
import pandas as pd
from scipy import stats
import sys

import os
import pickle

import seaborn as sns

try:
    import winsound
except ImportError:
    def playsound(frequency,duration):
        print("No sound available because Linux")
    
else:
    def playsound(frequency,duration):
        winsound.Beep(frequency,duration)



np.set_printoptions(suppress=True, precision=5) # disable scientific notation

# TO DO: # make plots of cp and cu depending on I_value

In [5]:
def Compute_piP(a,b,cp): 
    # compute pi_P given a,b, and cp =  p_v    np.dot( xi[:,0], np.multiply( xi[:,2], xi[:,1]))
    xi        = initialize_cities_array()
    
    # the following arrays do not change in the iteration, hence computed only once here
    
    CitiesToa = np.power(xi[:,0], a)
    CitiesTob1= np.power(xi[:,0], b+1)
    
    for i in range(N_iterations):
        Integral1 = 0
        Integral2 = 0
        Integral  = 0
        
        betaXpi = np.multiply(xi [:, 2] , xi[:,1])   #  \beta (dy) * \Pi _p (y) 
        

        Integral1 = np.dot(CitiesTob1,betaXpi)   # y ^{b+1} * \beta (dy) * \Pi _p (y) 
        Integral2 = np.dot(CitiesToa,  betaXpi)

        Exponent = Integral1* CitiesToa + Integral2 * CitiesTob1
        Exponent = kI * cp * Exponent
        xi [:, 2] = 1 - np.exp( - Exponent)
            
    return xi

In [6]:
def Compute_piP_old(a,b,cp): #deprecated
    # compute pi_P given a,b, and cp =  p_v    np.dot( xi[:,0], np.multiply( xi[:,2], xi[:,1]))
    xi        = initialize_cities_array()
    
    for i in range(N_iterations):
        Integral1 = 0
        Integral2 = 0
        Integral  = 0
        for j in range(np.shape(xi)[0]):
            Integral1 = Integral1 +  (xi[j,0]) ** (b+1) * xi [j, 2] * xi[j,1]   # y ^{b+1} * \beta (dy) * \Pi _p (y) 
            Integral2 = Integral2 +  (xi[j,0]) ** (a)   * xi [j, 2] * xi[j,1]

        for j in range(np.shape(xi)[0]):
            x = xi[j,0]
            Exponent = kI * cp * x ** a * Integral1 +  kI * cp * x ** (b+1) * Integral2
            #print(Exponent)
            #print(str(c) + ",  " + str(x)+ ",  " + str(Integral1)+ ",  " + str(Integral2))
            xi [j, 2] = 1 - math.exp( - Exponent)
            
    return xi

In [7]:
def Compute_piU(a,b,cu):
    # compute pi_U given a,b, and cu =  L_v 
    xi        = initialize_cities_array()
    
    # the following arrays do not change in the iteration, hence computed only once here
    CitiesToa1= np.power(xi[:,0], a-1)
    CitiesTob = np.power(xi[:,0], b)
    CitiesToa = np.power(xi[:,0], a)
    CitiesTob1= np.power(xi[:,0], b+1)
    
    for i in range(N_iterations):
        Integral1 = 0
        Integral2 = 0
        Integral  = 0
        
        betaXpi = np.multiply(xi [:, 3] , xi[:,1])   #  \beta (dy) * \Pi _u (y)
        

        Integral1 = np.dot(CitiesTob, betaXpi)   # y ^{b} * \beta (dy) * \Pi _u (y) 
        Integral2 = np.dot(CitiesToa1, betaXpi)
        
        Exponent = Integral1* CitiesToa + Integral2 * CitiesTob1
        Exponent = kI * cu * Exponent
        xi [:, 3] = 1 - np.exp( - Exponent)

            
    return xi 

In [8]:
def Compute_piU_old(a,b,cu): #deprecated
    # compute pi_U given a,b, and cu =  L_v 
    xi        = initialize_cities_array()
    
    for i in range(N_iterations):
        Integral1 = 0
        Integral2 = 0
        Integral  = 0
        for j in range(np.shape(xi)[0]):
            Integral1 = Integral1 +  (xi[j,0]) ** (b) * xi [j, 3] * xi[j,1]   # y ^{b} * \beta (dy) * \Pi _u (y) 
            Integral2 = Integral2 +  (xi[j,0]) ** (a-1)   * xi [j, 3] * xi[j,1]

        for j in range(np.shape(xi)[0]):
            x = xi[j,0]
            Exponent = kI * cu * x ** a * Integral1 + kI * cu * x ** (b+1) * Integral2
            #print(Exponent)
            #print(str(c) + ",  " + str(x)+ ",  " + str(Integral1)+ ",  " + str(Integral2))
            xi [j, 3] = 1 - math.exp( - Exponent)
            
    return xi 

In [9]:
# compute Ip given a,b, and cp =  p_v 

def Ip_function(a,b, cp):
    
    xi        = Compute_piP(a,b,cp)
    betaXpi = np.multiply(xi [:, 2] , xi[:,1])   #  \beta (dy) * \Pi _p (y) 
    Ip_value = np.dot(xi[:,0], betaXpi)   # y  * \beta (dy) * \Pi _p (y) 
    
    Ip_value = cp * Ip_value
    
    return Ip_value
    
    

    

In [10]:
# compute Ip given a,b, and cp =  p_v 

def Ip_function_old(a,b, cp): #deprecated
    xi        = Compute_piP(a,b,cp)

            
    Ip_value = 0
    for j in range(np.shape(xi)[0]):
        Ip_value = Ip_value + xi[j,0]* xi[j,2] * xi[j,1] 
    
    Ip_value = cp * Ip_value
    
    return Ip_value
    
    

    

In [11]:
# compute Iu given a,b,   cu =   L_v

def Iu_function(a,b, cu):
    
    
    xi        = Compute_piU(a,b,cu)
    Iu_value = np.dot(xi [:, 3] , xi[:,1])   #  \beta (dy) * \Pi _p (y)
    Iu_value = cu * Iu_value
    
    return Iu_value

In [12]:
# compute Iu given a,b,   cu =   L_v, deprecated

def Iu_function_old(a,b, cu): #deprecated
    
    xi = Compute_piU(a,b,cu)
       
    Iu_value = 0
    for j in range(np.shape(xi)[0]):
        Iu_value = Iu_value +  xi[j,3] * xi[j,1] 
    
    Iu_value = cu * Iu_value
    
    return Iu_value

In [13]:
 # compute Q_p given a,b, and cp =  p_v 

def Qp_function(a,b, cp):
    
    xi        = Compute_piP(a,b,cp)
   
    Qp_value = 0
    for j in range(np.shape(xi)[0]):
        Qp_value = Qp_value + xi[j,0]* xi[j,2] * xi[j,1]  # y * pi _p (y) * beta(y)
    
    return Qp_value
    

In [14]:
# compute Q_u given a,b, and cu =  L_v 

def Qu_function(a,b, cu):
    
    xi        = Compute_piU(a,b,cu)
   
    Qu_value = 0
    for j in range(np.shape(xi)[0]):
        Qu_value = Qu_value + xi[j,0]* xi[j,3] * xi[j,1]  # y * pi _u (y) * beta(y)
    
    return Qu_value

In [15]:
def Compute_cp(a,b,Q_value, Verbose = True):  # computes cp so that Qp_function(a,b,cp) = Q_value
    
    if Q_value > Q_max_possible_value - 0.1:
        raise ValueError("Q_value is too high!")
        
    cp = 1.9619 * 0.000001 * math.exp(0.03*np.random.normal())  #first guess
    This_should_get_to_Q_value = 0.8
    delta_c = cp * 0.04 * math.exp(0.011*np.random.normal())
    
    iteration_number = 0
    
    while abs(This_should_get_to_Q_value  - Q_value) > 0.1:
        iteration_number = iteration_number + 1
        if This_should_get_to_Q_value  - Q_value > 0.1:
            cp = cp - delta_c
        elif This_should_get_to_Q_value  - Q_value < -0.1:
            cp = cp + delta_c 
            
        if (This_should_get_to_Q_value - 1)/ (Q_value + 11) > 2.2:
            cp = cp * 0.77
            delta_c = cp * 0.04 * math.exp(0.05*np.random.normal())
        
        if (This_should_get_to_Q_value + 8)/ (Q_value ) <  0.42:
            cp = cp * 1.32
            delta_c = cp * 0.04 * math.exp(0.05*np.random.normal())
            
        if (Q_max_possible_value - This_should_get_to_Q_value )/ (Q_max_possible_value - Q_value + 11 ) > 2.2:
            cp = cp * 1.32
            delta_c = cp * 0.04 * math.exp(0.05*np.random.normal())
            
        if (Q_max_possible_value - This_should_get_to_Q_value + 8 )/ (Q_max_possible_value - Q_value ) < 0.42:
            cp = cp * 0.77
            delta_c = cp * 0.04 * math.exp(0.05*np.random.normal())
            
    
        delta_c = delta_c * 0.95
    
        This_should_get_to_Q_value = Qp_function(a, b, cp)
        if Verbose == True:
            if  iteration_number %137 == 4:
                print("cp = " + str(cp))
                print("This should be " + str(Q_value) + ": " + str(This_should_get_to_Q_value))
    
        if iteration_number % 200 == 198:
            delta_c = cp * 0.04 * math.exp(0.07*np.random.normal())
    if Verbose == True:
        print("cp = " + str(cp))
        print("This should be " +str(Q_value)+  ": " + str(This_should_get_to_Q_value))
        
    return cp

In [16]:
def Compute_cu(a,b,Q_value, Verbose = True):  # computes cu so that Qu_function(a,b,cu) = Q_value
    
    if Q_value > Q_max_possible_value - 0.1:
        raise ValueError("Q_value is too high!")
    
    cu = 0.52057344248  * math.exp(0.01*np.random.normal())  #first guess
    This_should_get_to_Q_value = 0.8
    delta_c = cu * 0.04 * math.exp(0.011*np.random.normal())
    
    iteration_number = 0
    
    while abs(This_should_get_to_Q_value  - Q_value) > 0.1:
        iteration_number = iteration_number + 1
        if This_should_get_to_Q_value  - Q_value > 0.1:
            cu = cu - delta_c
        elif This_should_get_to_Q_value  - Q_value < -0.1:
            cu = cu + delta_c 
            
        if (This_should_get_to_Q_value - 1)/ (Q_value + 11) > 2.2:
            cu = cu * 0.77
            delta_c = cu * 0.04 * math.exp(0.05*np.random.normal())
        
        if (This_should_get_to_Q_value + 8)/ (Q_value ) <  0.42:
            cu = cu * 1.32
            delta_c = cu * 0.04 * math.exp(0.05*np.random.normal())
            
        if (Q_max_possible_value - This_should_get_to_Q_value )/ (Q_max_possible_value - Q_value + 11 ) > 2.2:
            cu = cu * 1.32
            delta_c = cu * 0.04 * math.exp(0.05*np.random.normal())
            
        if (Q_max_possible_value - This_should_get_to_Q_value + 8 )/ (Q_max_possible_value - Q_value ) < 0.42:
            cu = cu * 0.77
            delta_c = cu * 0.04 * math.exp(0.05*np.random.normal())
    
        delta_c = delta_c * 0.95
    
        This_should_get_to_Q_value = Qu_function(a, b, cu)
        if Verbose == True:
            if  iteration_number %137 == 4:
                print("cu = " + str(cu))
                print("This should be " + str(Q_value) + ": " + str(This_should_get_to_Q_value))
    
        if iteration_number % 200 == 198:
            delta_c = cu * 0.04 * math.exp(0.07*np.random.normal())
    if Verbose == True:
        print("cu = " + str(cu))
        print("This should be " +str(Q_value)+  ": " + str(This_should_get_to_Q_value))
        
    return cu

In [17]:
def Compute_piU_and_piP(a,b, Q_value): 
    #for a given a,b, Q_value compute cp, cu, so that Qp = Qu = Q_value. Returns eta, 
    # the array consisting of 4 columns: zeroth column is city size y, first is beta(y), second is pi_p(y), third  pi_u(y)
    
    cp = Compute_cp(a,b,Q_value, Verbose = False)
    cu = Compute_cu(a,b,Q_value, Verbose = False)
    
    
    eta = initialize_cities_array()
    
    # recompute  pi_p(y) with the correct cp
    for i in range(N_iterations):
        Integral1 = 0
        Integral2 = 0
        Integral  = 0
        for j in range(np.shape(eta)[0]):
            Integral1 = Integral1 +  (eta[j,0]) ** (b+1) * eta [j, 2] * eta[j,1]   # y ^{b+1} * \beta (dy) * \Pi _p (y) 
            Integral2 = Integral2 +  (eta[j,0]) ** (a)   * eta [j, 2] * eta[j,1]

        for j in range(np.shape(eta)[0]):
            x = eta[j,0]
            Exponent = kI * cp * x ** a * Integral1 +  kI * cp * x ** (b+1) * Integral2
            #print(Exponent)
            #print(str(c) + ",  " + str(x)+ ",  " + str(Integral1)+ ",  " + str(Integral2))
            eta [j, 2] = 1 - math.exp( - Exponent)
    
    # recompute  pi_u(y) with the correct cp
    for i in range(N_iterations):
        Integral1 = 0
        Integral2 = 0
        Integral  = 0
        for j in range(np.shape(eta)[0]):
            Integral1 = Integral1 +  (eta[j,0]) ** (b) * eta [j, 3] * eta[j,1]   # y ^{b} * \beta (dy) * \Pi _u (y) 
            Integral2 = Integral2 +  (eta[j,0]) ** (a-1)   * eta [j, 3] * eta[j,1]

        for j in range(np.shape(eta)[0]):
            x = eta[j,0]
            Exponent = kI * cu * x ** a * Integral1 +  kI * cu * x ** (b+1) * Integral2
            #print(Exponent)
            #print(str(c) + ",  " + str(x)+ ",  " + str(Integral1)+ ",  " + str(Integral2))
            eta [j, 3] = 1 - math.exp( - Exponent)
            
    
    return [eta, cp, cu]

In [18]:
def Compute_Is(a,b,Q_value): #computes values I_p and I_q given a,b, and Q_value
    
    # get the correct cities array with correct probabilities
    eta, cp, cu = Compute_piU_and_piP(a,b, Q_value)
    
    
    Ip_value = 0
    for j in range(np.shape(eta)[0]):
        Ip_value = Ip_value + eta[j,0]* eta[j,2] * eta[j,1] 
    
    Ip_value = cp * Ip_value

    Iu_value = 0
    for j in range(np.shape(eta)[0]):
        Iu_value = Iu_value +  eta[j,3] * eta[j,1] 
    
    Iu_value = cu * Iu_value
 
     
    return [Ip_value, Iu_value]

In [19]:
def maxQ_value(): #maximal possilbe Q _value, corresponds to pi = 1
    xi  = initialize_cities_array()
    
    maxQ_value = 0
    for j in range(np.shape(xi)[0]):
        maxQ_value = maxQ_value + xi[j,0] * xi[j,1] # y * beta(y)
        
    return maxQ_value
        